# TiTiler-PgSTAC Demo


For this demo I'm going to use https://github.com/developmentseed/eoAPI

eoAPI let's you deploy a PgSTAC database + a STAC + a TiTiler-PGSTAC services


![](https://user-images.githubusercontent.com/10407788/129632832-fe6f7d80-4d87-463a-b0cc-305423a8489e.png)

In [14]:
import json
import httpx

from folium import Map, TileLayer, GeoJson

from geojson_pydantic import Feature, Polygon


raster_endpoint = "http://127.0.0.1:8082"
stac_endpoint = "http://127.0.0.1:8081"

In [9]:
# Before the demo I've created a collections and some items using 
# https://github.com/developmentseed/eoAPI/tree/master/demo/noaa

r = httpx.get(f"{stac_endpoint}/collections").json()

print(json.dumps(r, indent=4))

{
    "collections": [
        {
            "id": "noaa-emergency-response",
            "links": [
                {
                    "rel": "items",
                    "type": "application/geo+json",
                    "href": "http://127.0.0.1:8081/collections/noaa-emergency-response/items"
                },
                {
                    "rel": "parent",
                    "type": "application/json",
                    "href": "http://127.0.0.1:8081/"
                },
                {
                    "rel": "root",
                    "type": "application/json",
                    "href": "http://127.0.0.1:8081/"
                },
                {
                    "rel": "self",
                    "type": "application/json",
                    "href": "http://127.0.0.1:8081/collections/noaa-emergency-response"
                }
            ],
            "title": "NOAA Emergency Response Imagery",
            "extent": {
                "spatial": {
 

In [13]:
r = httpx.get(f"{stac_endpoint}/collections/noaa-emergency-response/items").json()

print(json.dumps(r["context"], indent=4))

{
    "limit": 10,
    "matched": 163,
    "returned": 10
}


In [15]:
# bounds of the noaa-eri-nashville2020.json items
bounds = (-87.0251, 36.0999, -85.4249, 36.2251)

poly = Polygon.from_bounds(*bounds)
geojson = Feature(geometry=poly).dict(exclude_none=True)

m = Map(
    tiles="OpenStreetMap",
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=8
)

geo_json = GeoJson(
    data=geojson,
    style_function=lambda x: {
        'opacity': 1, 'dashArray': '1', 'fillOpacity': 0, 'weight': 1
    },
)
geo_json.add_to(m)
m

In [18]:
r = httpx.get(f"{stac_endpoint}/collections/noaa-emergency-response/items").json()
item = r["features"][0]

print(json.dumps(item, indent=4))

{
    "id": "20200307aC0862400w361330",
    "bbox": [
        -86.4001,
        36.1999,
        -86.3749,
        36.2251
    ],
    "type": "Feature",
    "links": [
        {
            "rel": "collection",
            "type": "application/json",
            "href": "http://127.0.0.1:8081/collections/noaa-emergency-response"
        },
        {
            "rel": "parent",
            "type": "application/json",
            "href": "http://127.0.0.1:8081/collections/noaa-emergency-response"
        },
        {
            "rel": "root",
            "type": "application/json",
            "href": "http://127.0.0.1:8081/"
        },
        {
            "rel": "self",
            "type": "application/geo+json",
            "href": "http://127.0.0.1:8081/collections/noaa-emergency-response/items/20200307aC0862400w361330"
        }
    ],
    "assets": {
        "cog": {
            "href": "https://noaa-eri-pds.s3.us-east-1.amazonaws.com/2020_Nashville_Tornado/20200307a_RGB/2020030

In [22]:
info = httpx.get(
    f"{raster_endpoint}/collections/{item['collection']}/items/{item['id']}/info",
    params={
        "assets": "cog",
    }
).json()

print(json.dumps(info, indent=4))

{
    "cog": {
        "bounds": [
            -86.4001,
            36.1999,
            -86.3749,
            36.2251
        ],
        "minzoom": 13,
        "maxzoom": 20,
        "band_metadata": [
            [
                "b1",
                {}
            ],
            [
                "b2",
                {}
            ],
            [
                "b3",
                {}
            ]
        ],
        "band_descriptions": [
            [
                "b1",
                ""
            ],
            [
                "b2",
                ""
            ],
            [
                "b3",
                ""
            ]
        ],
        "dtype": "uint8",
        "nodata_type": "Mask",
        "colorinterp": [
            "red",
            "green",
            "blue"
        ],
        "height": 18681,
        "driver": "GTiff",
        "overviews": [
            2,
            4,
            8,
            16,
            32,
            64,
     

In [24]:
info = httpx.get(
    f"{raster_endpoint}/collections/{item['collection']}/items/{item['id']}/info",
    params={
        "assets": "cog",
    }
).json()["cog"]

r = httpx.get(
    f"{raster_endpoint}/collections/{item['collection']}/items/{item['id']}/tilejson.json",
    params = (
        ("assets", "cog"),
        ("minzoom", info["minzoom"]),
        ("maxzoom", info["maxzoom"]),
    )
).json()
print(r)

m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=r["minzoom"]
)

aod_layer = TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="Maxar"
)
aod_layer.add_to(m)
m

{'tilejson': '2.2.0', 'version': '1.0.0', 'scheme': 'xyz', 'tiles': ['http://127.0.0.1:8082/collections/noaa-emergency-response/items/20200307aC0862400w361330/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=cog'], 'minzoom': 13, 'maxzoom': 20, 'bounds': [-86.4001, 36.1999, -86.3749, 36.2251], 'center': [-86.38749999999999, 36.2125, 13]}


In [25]:
bounds = (-87.0251, 36.0999, -85.4249, 36.2251)

# STAC API /search request
search_request = {
    # Filter collection
    "collections": ["noaa-emergency-response"],
    # limit bounds of the known items (note: the bbox will also be used in the tilejson response)
    "bbox": bounds,
    "filter-lang": "cql-json",
}

r = httpx.post(
    f"{raster_endpoint}/mosaic/register", json=search_request,
).json()

searchid = response["searchid"]

print(json.dumps(r, indent=4))

{
    "searchid": "6d436413d0eed760acc2f6bd16ca77a5",
    "links": [
        {
            "rel": "metadata",
            "type": "application/json",
            "href": "http://127.0.0.1:8082/mosaic/6d436413d0eed760acc2f6bd16ca77a5/info"
        },
        {
            "rel": "tilejson",
            "type": "application/json",
            "href": "http://127.0.0.1:8082/mosaic/6d436413d0eed760acc2f6bd16ca77a5/tilejson.json"
        }
    ]
}


In [29]:
r = httpx.get(
    f"{raster_endpoint}/mosaic/{searchid}/tilejson.json?assets=cog",
    params={
        "minzoom": 13,
        "maxzoom": 20,
    }
).json()
print(json.dumps(r, indent=4))

{
    "tilejson": "2.2.0",
    "name": "6d436413d0eed760acc2f6bd16ca77a5",
    "version": "1.0.0",
    "scheme": "xyz",
    "tiles": [
        "http://127.0.0.1:8082/mosaic/6d436413d0eed760acc2f6bd16ca77a5/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=cog"
    ],
    "minzoom": 13,
    "maxzoom": 20,
    "bounds": [
        -87.0251,
        36.0999,
        -85.4249,
        36.2251
    ],
    "center": [
        -86.225,
        36.162499999999994,
        13
    ]
}


In [31]:
bounds = r["bounds"]

m = Map(
    location=(r["center"][1], r["center"][0]),
    zoom_start=r["minzoom"]
)

poly = Polygon.from_bounds(*bounds)
geojson = Feature(geometry=poly).dict(exclude_none=True)

m = Map(
    tiles="OpenStreetMap",
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=13
)

geo_json = GeoJson(
    data=geojson,
    style_function=lambda x: {
        'opacity': 1, 'dashArray': '1', 'fillOpacity': 0, 'weight': 1
    },
)
geo_json.add_to(m)
m

aod_layer = TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="esri-noaa"
)
aod_layer.add_to(m)
m